In [1]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
    
#Práctica realizada por Javier Guillamón y Marino Pérez
  
class xorMLP(object):
    #0 0 | 0
    #0 1 | 1
    #1 0 | 1
    #1 1 | 0
    def __init__(self, learning_rate=0.2):
        self.learning_rate = learning_rate      
        self.y = [0,1,1,0]
        self.X = [[0,0],[0,1],[1,0],[1,1]]

        self.w02 = 2* np.random.random((1,1))-1
        self.w03 = 2* np.random.random((1,1))-1
        self.w12 = 2* np.random.random((1,1))-1
        self.w13 = 2* np.random.random((1,1))-1
        self.w23 = 2* np.random.random((1,1))-1
        self.wb2 = 2* np.random.random((1,1))-1
        self.wb3 = 2* np.random.random((1,1))-1
        self.Bias = 1
        
    #Metodo que recorre la red neuronal y devuelve su resultado    
    def feedFordward(self, X):
       self.x2 = self.sigmoide(self.w02*X[0] + self.w12*X[1] + self.Bias*self.wb2) #Salida de la capa oculta, Pesos*Entradas+Bias, usando la funcion sigmoide
       x3 = self.sigmoide(self.w23*self.x2 + self.w03*X[0] + self.w13*X[1] + self.Bias*self.wb3) #Salida de la capa final
       return x3
    
    #Funcion de activacion sigmoide
    def sigmoide(self,x,sigmoide=True):
        return 1 / (1 + np.exp(-x))
    
    #Metodo para saber si se han alcanzado los margenes de % de acierto
    def errorAlcanzado(self, yHat,i):
        value = 0
        if yHat >= 0.8: value = 1
        elif yHat <= 0.2: value = 0
        else: return False
        if value==self.y[i]: return True
        return False
    
    #Metodo para actualizar los valores ed los pesos
    def Backpropagation(self):
       i=0  #Contador de iteraciones para evitar bucles infinitos
       yAux= [0,0,0,0]  #Vector auxiliar que almacena los resultados obtenidos de y en una iteracion del bucle
       errorAlcanzado=False  #bool que nos indica si se han alcanzado los margenes de error
       while errorAlcanzado==False: 
          for j in range(len(self.X)):
            i+=1
            x0=self.X[j][0]
            x1=self.X[j][1]
            yHat = self.feedFordward([x0,x1])  
            yAux[j]=yHat
            error = yHat * (1 - yHat) * (self.y[j] - yHat) #Error de la capa de salida
            self.w03 +=self.learning_rate*error*self.X[j][0]
            self.w13 +=self.learning_rate*error*self.X[j][1]
            self.w23 +=self.learning_rate*error*self.x2
            self.wb3 +=self.learning_rate*error*self.Bias
            error2 = error * self.w23 * self.x2 * (1-self.x2) #Error de la capa oculta
            self.w12 +=self.learning_rate*error2*self.X[j][1]
            self.w02 +=self.learning_rate*error2*self.X[j][0]
            self.wb2 +=self.learning_rate*error2*self.Bias            
          for k in range(len(yAux)): #bucle para comprobar que todos 
            resultado = self.errorAlcanzado(yAux[k],k)
            if resultado==False: break
          errorAlcanzado = resultado
          if i==100000: errorAlcanzado = True #En el caso de que se hagan 100000 iteraciones se cierra el bucle
       print("Iteraciones: %d"%i)
    
    def fit(self):
        self.Backpropagation()
  
     
    def predict(self, x):
        """
        x = [x1, x2]
        """
        print(self.feedFordward(x))
        return '1' if (self.feedFordward(x)>=0.5) else '0' #Redondeamos los datos para mostrarlos en pantalla
      
class DeepMLP(object):
    def __init__(self, layers_size, learning_rate=0.):
        """
        ejemplos layers_size 
            [100, 50, 10] 100 neuronas de entrada, 50 de capa oculta 1 y 10 de salida
            [100, 50, 20, 10] 100 neuronas de entrada, 50 de capa oculta 1, 50 de capa oculta 2 y 10 de salida
            [100, 50, 20, 50, 10] etc.
        """  
        self.layers_size = layers_size
        self.learning_rate = learning_rate
        
        self.x = tf.placeholder(tf.float32,[None,self.layers_size[0]]) #Placeholder que almacenara una matriz[X,Y] que puede tener cualquier cantidad de valores en X, en Y tiene que tener tantos valores como se indiquen en la primera capa de layers_size
        self.y = tf.placeholder(tf.float32,[None,self.layers_size[-1]]) #Igual que el anterior, pero en Y almacena la cantidad de valores indicados en la ultima capa de layers_size
        #Un placeholder es una variable a la que no se le asignaran datos en otro momento
        self.w = [] #Lista que almacena todos los pesos de la red
        self.b = [] #Lista que almacena todos los Bias de la red
        for i in range(len(self.layers_size)-1):
          self.w.append(tf.Variable(2*np.random.random((self.layers_size[i],self.layers_size[i+1]))-1,dtype=tf.float32)) #inicializacion de los pesos con valores aleatorios de -1 a 1, se agrupan en matrices de tamaño nºnodosEntrada * nºnodosSalida 
          self.b.append(tf.Variable(2*np.random.random((1,self.layers_size[i+1]))-1,dtype=tf.float32)) #igual que los pesos pero en este caso solo se almacenan vectores de tamaño nºnodosSalida
        
        #Feedfordward de la red
        self.yHat = self.x
        for i in range(len(self.layers_size)-1): #Se realiza la operacion tantas veces como capas haya
          self.yHat = tf.matmul(self.yHat,self.w[i])+self.b[i] #salida de la capa = entrada*pesos+Bias
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,logits=self.yHat)) #Determina la perdida que tiene el modelo, nos dice como de ineficiente son las predicciones de la red, ademas le aplica directamente la funcion de activacion softmax
        self.train_step = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(cross_entropy) #Minimiza el crozz_entropy usando un descenso de gradiente
        
        self.sess = tf.InteractiveSession() #Crea una sesion de Tensorflow  
        self.sess.run(tf.initialize_all_variables()) #inicializa todas la varibles creadas en tensorflow 
        #en mi caso estoy usando tf.initialize_all_variables() y aque estoy con la version 0.8 de TensorFlow, en una version mas actual se puede cambiar por tf.global_variables_initializer()


    def fit(self, X):
        """
        X = entradas del conjunto de datos de entrenamiento, puede ser un batch o una sola tupla
        Y = salidas esperadas del conjunto de datos de entrenamiento, puede ser un batch o una sola tupla
        """
        porcentageAlcanzado = False #bool que nos indica si se han alcanzado el % de acierto dado
        i=0 #Contador de iteraciones para evitar bucles infinitos
        batchX, batchY = X.next_batch(100) #Se cogen un batch con 100 elementos aleatorios dentro de X y se dividen en datos de entrada batchX y datos esperados de salida batchY
        self.score(batchX,batchY) #Llama a la funcion score para inicializar las variables necesarias
        while porcentageAlcanzado == False:
          i+=1
          if i % 100==0: #cada 100 iteraciones se hace una actualizacion de la precision del entrenamiento 
            train_accuracy = self.accuracy.eval(feed_dict={self.x:batchX,self.y:batchY}) #Se evalua actualiza el valor de accuracy 
            print('step %d, training accuracy %g'%(i,train_accuracy))
            if train_accuracy >= 0.98: #Si la precision alcanza un valor dado paramos el entrenamiento
              porcentageAlcanzado = True
          self.sess.run(self.train_step,feed_dict={self.x:batchX,self.y:batchY}) #Se actualiza el valor de train_step
          if i >= 100000: #En el caso de llegar a 100000 iteraciones se para el entrenamiento
            porcentageAlcanzado = True
          batchX, batchY = X.next_batch(100) #Se coge el siguiente batch para usar en la siguiente iteracion
        print("Iteraciones: %d"%i)
          
    def score(self, X, Y):
        """
        X = entradas del conjunto de datos de testeo, puede ser un batch o una sola tupla
        Y = salidas esperadas del conjunto de datos de testeo, puede ser un batch o una sola tupla
        """
        correct_prediction = tf.equal(tf.argmax(self.yHat,1),tf.argmax(self.y,1)) #Se determina si la predcicion realizada en yHat es correcta. yHat e Y son dos arrays de 10 elementos, cada elemento dentro del array determinala probabilidad de que el numero a predecir sea el indice del array es decir, [0,0,1] el resultado seria 2. argmax coge el valor mas grande dentro del array, por lo que se comparan el valor de yHat con mas probabilidad frente al valor real de y
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) # la prediccion se saca cogiendo el array de booleanos anterior, se castea a float para poder hacer la media, ek resultado nos determina cual es la probabilidad de acertar 
        self.a =self.sess.run(self.accuracy, feed_dict={self.x:X,self.y:Y}) #Se ejecuta accuracy dentro de Tensorflow con los valores de testeo X e Y
        print('accuracy %g'%self.a)
        
if __name__ == '__main__':
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    np.random.seed(1)
    
    print("XOR:")
    xor= xorMLP(0.2)
    xor.fit()
    predict00 = xor.predict([0,0])
    predict10 = xor.predict([1,0])
    predict01 = xor.predict([0,1])
    predict11 = xor.predict([1,1])
    print("Predict 00: "+str(predict00))
    print("Predict 10: "+str(predict10))
    print("Predict 01: "+str(predict01))
    print("Predict 11: "+str(predict11))
    
    print("")
    #implementar un metodo para entrenar el DeepMLP, hay que intentar llegar a un 90 y pico % de aacierto
    #TODO MNIST TESTS
    print("DeepMLP:")
    dpml = DeepMLP([784,10,10,10],0.2)
    X_train = mnist.train
    dpml.fit(X_train)
    X_test = mnist.test.images
    Y_test = mnist.test.labels
    dpml.score(X_test,Y_test)
